In [1]:
import torch
import numpy as np
import os
import pickle
import argparse
import matplotlib.pyplot as plt
from copy import deepcopy
from tqdm import tqdm
from einops import rearrange

from constants import DT
from constants import PUPPET_GRIPPER_JOINT_OPEN
from utils import load_data # data functions
from utils import sample_box_pose, sample_insertion_pose # robot functions
from utils import compute_dict_mean, set_seed, detach_dict # helper functions
from policy import ACTPolicy, CNNMLPPolicy
from visualize_episodes import save_videos

from sim_env import BOX_POSE

import IPython
policy_class = 'ACT'
lr = 1e-4
kl_weight = 10
chunk_size = 100 
hidden_dim = 512
dim_feedforward = 3200
camera_names = ['cam_high', 'cam_low', 'cam_left_wrist', 'cam_right_wrist']
num_epochs = 8000
ckpt_dir = '/home/horowitzlab/project/act/ckpt_dir/real_peg_insertion'
episode_len = 300
state_dim = 12
task_name = 'real_peg_insertion'
seed = 0

onscreen_render = False
is_sim = False
temporal_agg = False

lr_backbone = 1e-5
backbone = 'resnet18'
if policy_class == 'ACT':
    enc_layers = 4
    dec_layers = 7
    nheads = 8
    policy_config = {'lr': lr,
                        'num_queries':chunk_size,
                        'kl_weight': kl_weight,
                        'hidden_dim': hidden_dim,
                        'dim_feedforward': dim_feedforward,
                        'lr_backbone': lr_backbone,
                        'backbone': backbone,
                        'enc_layers': enc_layers,
                        'dec_layers': dec_layers,
                        'nheads': nheads,
                        'camera_names': camera_names,
                        'ckpt_dir': ckpt_dir,
                        'policy_class': policy_class,
                        'task_name': task_name,
                        'seed': seed,
                        'num_epochs': num_epochs
                        }
elif policy_class == 'CNNMLP':
    policy_config = {'lr': lr, 'lr_backbone': lr_backbone, 'backbone' : backbone, 'num_queries': 1,
                        'camera_names': camera_names,}
else:
    raise NotImplementedError

config = {
    'num_epochs': num_epochs,
    'ckpt_dir': ckpt_dir,
    'episode_len': episode_len,
    'state_dim': state_dim,
    'lr': lr,
    'policy_class': policy_class,
    'onscreen_render': onscreen_render,
    'policy_config': policy_config,
    'task_name': task_name,
    'seed': seed,
    'temporal_agg': temporal_agg,
    'camera_names': camera_names,
    'real_robot': not is_sim
}


In [4]:
def make_policy(policy_class, policy_config):
    if policy_class == 'ACT':
        policy = ACTPolicy(policy_config)
    elif policy_class == 'CNNMLP':
        policy = CNNMLPPolicy(policy_config)
    else:
        raise NotImplementedError
    return policy

def eval_bc(config, ckpt_name, save_episode=True):
    set_seed(1000)
    ckpt_dir = config['ckpt_dir']
    state_dim = config['state_dim']
    real_robot = config['real_robot']
    policy_class = config['policy_class']
    onscreen_render = config['onscreen_render']
    policy_config = config['policy_config']
    camera_names = config['camera_names']
    max_timesteps = config['episode_len']
    task_name = config['task_name']
    temporal_agg = config['temporal_agg']
    onscreen_cam = 'angle'

    # load policy and stats
    ckpt_path = os.path.join(ckpt_dir, ckpt_name)
    policy = make_policy(policy_class, policy_config)
    loading_status = policy.load_state_dict(torch.load(ckpt_path))
    print(loading_status)
    policy.cuda()
    policy.eval()
    print(f'Loaded: {ckpt_path}')
    stats_path = os.path.join(ckpt_dir, f'dataset_stats.pkl')
    with open(stats_path, 'rb') as f:
        stats = pickle.load(f)

    pre_process = lambda s_qpos: (s_qpos - stats['qpos_mean']) / stats['qpos_std']
    post_process = lambda a: a * stats['action_std'] + stats['action_mean']

In [5]:
eval_bc(config,"/home/horowitzlab/project/act/ckpt_dir/real_peg_insertion/policy_last.ckpt" )

args_override: {'lr': 0.0001, 'num_queries': 100, 'kl_weight': 10, 'hidden_dim': 512, 'dim_feedforward': 3200, 'lr_backbone': 1e-05, 'backbone': 'resnet18', 'enc_layers': 4, 'dec_layers': 7, 'nheads': 8, 'camera_names': ['cam_high', 'cam_low', 'cam_left_wrist', 'cam_right_wrist'], 'ckpt_dir': '/home/horowitzlab/project/act/ckpt_dir/real_peg_insertion', 'policy_class': 'ACT', 'task_name': 'real_peg_insertion', 'seed': 0, 'num_epochs': 8000}


usage: DETR training and evaluation script [-h] [--lr LR]
                                           [--lr_backbone LR_BACKBONE]
                                           [--batch_size BATCH_SIZE]
                                           [--weight_decay WEIGHT_DECAY]
                                           [--epochs EPOCHS]
                                           [--lr_drop LR_DROP]
                                           [--clip_max_norm CLIP_MAX_NORM]
                                           [--backbone BACKBONE] [--dilation]
                                           [--position_embedding {sine,learned}]
                                           [--camera_names CAMERA_NAMES]
                                           [--enc_layers ENC_LAYERS]
                                           [--dec_layers DEC_LAYERS]
                                           [--dim_feedforward DIM_FEEDFORWARD]
                                           [--hidden_dim HIDDEN_DIM]
            

SystemExit: 2

In [ ]:
%tb

SystemExit: 2